Scenario

You are working as an analyst with this internet service provider. You are provided with this historical data about your company's customers and their churn trends. Your task is to build a machine learning model that will help the company identify customers that are more likely to default/churn and thus prevent losses from such customers.

Instructions

In this lab, we will first take a look at the degree of imbalance in the data and correct it using the techniques we learned on the class. Here is the list of steps to be followed (building a simple model without balancing the data):

Round 1

 Import the required libraries and modules that you would need.

In [35]:
import pandas as pd
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

Read that data into Python and call the dataframe churnData.

In [36]:
churnData = pd.read_csv("/Users/jon/Documents/IRONHACK/DAFT_0116/module_3/LAB Imbalanced Classification Task/DATA_Customer-Churn.txt")

In [37]:
churnData.shape

(7043, 16)

Check the datatypes of all the columns in the data.You would see that the column TotalCharges is object type. Convert this column into numeric type using pd.to_numeric function.


In [38]:
churnData.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [39]:
churnData["TotalCharges"] = pd.to_numeric(churnData["TotalCharges"], errors='coerce')

Check for null values in the dataframe. Replace the null values.

In [40]:
churnData.isnull().sum()

gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [41]:
churnData.fillna(churnData.median(), inplace=True)
churnData.isnull().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

Use the following features: tenure, SeniorCitizen, MonthlyCharges and TotalCharges:

Split the data into a training set and a test set.

Scale the features either by using MinMaxScaler or a standard scaler.

In [42]:
churnData.loc[(churnData['Churn']=="No"), 'Churn'] = '0'
churnData.loc[(churnData['Churn']=="Yes"), 'Churn'] = '1'



In [67]:
X = churnData[["tenure", "SeniorCitizen", "MonthlyCharges", "TotalCharges"]]
y = churnData["Churn"]

In [68]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=11)

In [104]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# transform data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test) #No need to use fit since we should normalize the training set and then use the same normalization steps on the test set


ROUND 2

(Optional) Fit a logistic Regression model on the training data.

In [89]:
log_model = LogisticRegression() 

#trans=PowerTransformer() #Optional transformation
#trans.fit(X_train_scaled)
#X_train_mod = trans.transform(X_train_scaled)
#X_test_mod  = trans.transform(X_test_scaled)
log_model.fit(X_train_scaled, y_train)
log_y_pred_test = log_model.predict(X_test_scaled) # predict test
log_y_pred_train=log_model.predict(X_train_scaled) # predict train (for sanity checks)


Fit a Knn Classifier (NOT KnnRegressor please!)model on the training data.

In [96]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=2,weights='uniform') # declare we're using knn classification model
knn_model.fit(X_train_scaled, y_train) # train model
knn_y_pred = knn_model.predict(X_test_scaled) # predict test
knn_y_pred_train=knn_model.predict(X_train_scaled) # predict train (for sanity checks)
#model.predict_proba(inputdata)

Fit a Decision Tree Classifier on the training data.

In [97]:
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier(max_depth=3) #Comes from sklearn.tree

dt_model.fit(X_train_scaled, y_train)

y_pred_train_dt = dt_model.predict(X_train_scaled)
y_pred_test_dt = dt_model.predict(X_test_scaled)

Compare the accuracy, precision, recall for the previous models on both the train and test sets.

In [98]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt


In [99]:
performance_log = pd.DataFrame({'Error_metric': ['Accuracy','Precision','Recall'],
                               'Train': [accuracy_score(y_train, log_y_pred_train),
                                         precision_score(y_train, log_y_pred_train, pos_label='1'),
                                         recall_score(y_train, log_y_pred_train, pos_label='1')],
                               'Test': [accuracy_score(y_test, log_y_pred_test),
                                        precision_score(y_test, log_y_pred_test, pos_label='1'),
                                        recall_score(y_test, log_y_pred_test, pos_label='1')]})

display(performance_log)

performance_knn = pd.DataFrame({'Error_metric': ['Accuracy','Precision','Recall'],
                               'Train': [accuracy_score(y_train, knn_y_pred_train),
                                         precision_score(y_train, knn_y_pred_train, pos_label='1'),
                                         recall_score(y_train, knn_y_pred_train, pos_label='1')],
                               'Test': [accuracy_score(y_test, knn_y_pred),
                                        precision_score(y_test, knn_y_pred, pos_label='1'),
                                        recall_score(y_test, knn_y_pred, pos_label='1')]})

display(performance_knn)

performance_dt = pd.DataFrame({'Error_metric': ['Accuracy','Precision','Recall'],
                               'Train': [accuracy_score(y_train, y_pred_train_dt),
                                         precision_score(y_train, y_pred_train_dt, pos_label='1'),
                                         recall_score(y_train, y_pred_train_dt, pos_label='1')],
                               'Test': [accuracy_score(y_test, y_pred_test_dt),
                                        precision_score(y_test, y_pred_test_dt, pos_label='1'),
                                        recall_score(y_test, y_pred_test_dt, pos_label='1')]})

display(performance_dt)

,Error_metric,Train,Test
0,Accuracy,0.788782,0.801987
1,Precision,0.645975,0.692607
2,Recall,0.446680,0.470899


,Error_metric,Train,Test
0,Accuracy,0.865460,0.766501
1,Precision,0.989319,0.636872
2,Recall,0.496982,0.301587


,Error_metric,Train,Test
0,Accuracy,0.784523,0.784244
1,Precision,0.647184,0.672897
2,Recall,0.408451,0.380952


Round 3

apply K-fold cross validation on your models built before,  and check the model score. Note: So far we have not balanced the data.

In [100]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict


In [101]:
#Logistic
log_scores=cross_val_score(log_model, X_train_scaled, y_train, cv=5) #cv = k (number of folds)
print("Logistic Model \nCross validation scores: ", log_scores)
print("Score stats: %0.2f accuracy with a standard deviation of %0.2f" % (log_scores.mean(), log_scores.std()))


#Knn
knn_scores=cross_val_score(knn_model, X_train_scaled, y_train, cv=5) #cv = k (number of folds)
print("\nKNN Model \nCross validation scores: ", knn_scores)
print("Score stats: %0.2f accuracy with a standard deviation of %0.2f" % (knn_scores.mean(), knn_scores.std()))

#DT
dt_scores=cross_val_score(dt_model, X_train_scaled, y_train, cv=5) #cv = k (number of folds)
print("\nDecision Tree Model \nCross validation scores: ", dt_scores)
print("Score stats: %0.2f accuracy with a standard deviation of %0.2f" % (dt_scores.mean(), dt_scores.std()))

Logistic Model 
Cross validation scores:  [0.7755102  0.80124224 0.79148181 0.79148181 0.78863233]
Score stats: 0.79 accuracy with a standard deviation of 0.01

KNN Model 
Cross validation scores:  [0.7675244  0.76397516 0.7675244  0.78438332 0.77442274]
Score stats: 0.77 accuracy with a standard deviation of 0.01

Decision Tree Model 
Cross validation scores:  [0.77196096 0.78881988 0.78438332 0.77639752 0.78952043]
Score stats: 0.78 accuracy with a standard deviation of 0.01


Round 4

fit a Random forest Classifier on the data and compare the accuracy.
tune the hyper parameters with Gridsearch and check the results. retrain the final mode with the best parameters found.


In [113]:
from sklearn.ensemble import RandomForestClassifier
RAND_STATE = 42 # for reproducible shuffling

rfc_ops = {"max_depth":6, #Controls depth for all trees
           "min_samples_leaf":20,
           "max_features":None,
           "n_estimators":100, #number of trees
           "bootstrap":True, #You can disable this if you don't like
           "oob_score":True, #Out of bag score. Use records that weren't already used. Linked to bootstrap
           "random_state":RAND_STATE}

clf = RandomForestClassifier(**rfc_ops)

        #max_depth=6,min_samples_leaf=20,max_features=None,n_estimators=100,
         #                    bootstrap=True,oob_score=True, random_state=RAND_STATE)
clf.fit(X_train_scaled, y_train)
print("train prediction accuracy score: %.2f" %(clf.score(X_train_scaled, y_train)))
print("test prediction accuracy score: %.2f"  %(clf.score(X_test_scaled, y_test)))

train prediction accuracy score: 0.81
test prediction accuracy score: 0.79


In [124]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [50, 100,500],
    'min_samples_split': [2, 4],
    'min_samples_leaf' : [1, 2],
    'max_features': ['sqrt']
    ##'max_samples' : ['None', 0.5],
    ##'max_depth':[3,5,10],
    ## 'bootstrap':[True,False]
    }
clf2 = RandomForestClassifier(random_state=RAND_STATE)

In [125]:
grid_search = GridSearchCV(clf2, param_grid, cv=5,return_train_score=True,n_jobs=-1)

In [126]:
grid_search.fit(X_train_scaled,y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_features': ['sqrt'], 'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 4],
                         'n_estimators': [50, 100, 500]},
             return_train_score=True)

In [127]:
best_params = grid_search.best_params_ #To check the best set of parameters returned
best_params

{'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 100}

In [128]:
from sklearn.model_selection import cross_val_score
import numpy as np

clf3 = RandomForestClassifier(random_state=RAND_STATE, **best_params)
cross_val_scores_RF = cross_val_score(clf3, X_test_scaled, y_test, cv=5)
print(np.mean(cross_val_scores_RF))
#accuracy becomes worse but stability will be higher.

0.7650589333939073


Managing imbalance in the dataset

Check for the imbalance.
Use the resampling strategies used in class for upsampling and downsampling to create a balance between the two classes.
Each time fit the model and check the accuracy of the model.

In [151]:
df = churnData
category_0 = df[df["Churn"] == "0"] # negative cases (majority)
category_1 = df[df["Churn"] == "1"] # positive cases (minority)
df["Churn"].shape

print(category_0.shape)
print(category_1.shape)

(5174, 16)
(1869, 16)


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,0
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.50,0
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,1
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,0
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,1


In [152]:
c1_len=1869
# downsample the majority class to the size of the positive class using pandas sample method
category_0_down = category_0.sample(c1_len)
print(category_0_down.shape)
print(category_1.shape)

(1869, 16)
(1869, 16)


In [154]:
# reassemble the data
data = pd.concat([category_0_down, category_1], axis=0)
# shuffle the data
data = data.sample(frac=1) # frac specifies ratio of the shuffled output to the input size. for frac=1 the number of rows is unchanged
data['Churn'].value_counts()

1    1869
0    1869
Name: Churn, dtype: int64

In [155]:
X_data = data[["tenure", "SeniorCitizen", "MonthlyCharges", "TotalCharges"]]
y_data = data["Churn"]

X_data_train, X_data_test, y_data_train, y_data_test = train_test_split(X_data, y_data, test_size=0.20, random_state=11)

scaler = StandardScaler()
# transform data
X_data_train = scaler.fit_transform(X_data_train)
X_data_test = scaler.transform(X_data_test) #No need to use fit since we should normalize the training set and then use the same normalization steps on the test set


In [158]:
log_model_data = LogisticRegression() 
#trans=PowerTransformer() #Optional transformation
#trans.fit(X_train_scaled)
#X_train_mod = trans.transform(X_train_scaled)
#X_test_mod  = trans.transform(X_test_scaled)
log_model.fit(X_data_train, y_data_train)
log_y_pred_test_data = log_model.predict(X_data_test) # predict test
log_y_pred_train_data = log_model.predict(X_data_train) # predict train (for sanity checks)

from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=2,weights='uniform') # declare we're using knn classification model
knn_model.fit(X_data_train, y_data_train) # train model
knn_y_pred_data = knn_model.predict(X_data_test) # predict test
knn_y_pred_train_data = knn_model.predict(X_data_train) # predict train (for sanity checks)
#model.predict_proba(inputdata)

from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier(max_depth=3) #Comes from sklearn.tree
dt_model.fit(X_data_train, y_data_train)
y_pred_train_dt_data = dt_model.predict(X_data_train)
y_pred_test_dt_data = dt_model.predict(X_data_test)

In [160]:
print("Logistic Regression Classifier with DownSampling")
performance_log_data = pd.DataFrame({'Error_metric': ['Accuracy','Precision','Recall'],
                               'Train': [accuracy_score(y_data_train, log_y_pred_train_data),
                                         precision_score(y_data_train, log_y_pred_train_data, pos_label='1'),
                                         recall_score(y_data_train, log_y_pred_train_data, pos_label='1')],
                               'Test': [accuracy_score(y_data_test, log_y_pred_test_data),
                                        precision_score(y_data_test, log_y_pred_test_data, pos_label='1'),
                                        recall_score(y_data_test, log_y_pred_test_data, pos_label='1')]})

display(performance_log_data)

print("\nKNN Classifier with DownSampling")

performance_knn_data = pd.DataFrame({'Error_metric': ['Accuracy','Precision','Recall'],
                               'Train': [accuracy_score(y_data_train, knn_y_pred_train_data),
                                         precision_score(y_data_train, knn_y_pred_train_data, pos_label='1'),
                                         recall_score(y_data_train, knn_y_pred_train_data, pos_label='1')],
                               'Test': [accuracy_score(y_data_test, knn_y_pred_data),
                                        precision_score(y_data_test, knn_y_pred_data, pos_label='1'),
                                        recall_score(y_data_test, knn_y_pred_data, pos_label='1')]})

display(performance_knn_data)

print("\nDecision Tree Classifier with DownSampling")

performance_dt_data = pd.DataFrame({'Error_metric': ['Accuracy','Precision','Recall'],
                               'Train': [accuracy_score(y_data_train, y_pred_train_dt_data),
                                         precision_score(y_data_train, y_pred_train_dt_data, pos_label='1'),
                                         recall_score(y_data_train, y_pred_train_dt_data, pos_label='1')],
                               'Test': [accuracy_score(y_data_test, y_pred_test_dt_data),
                                        precision_score(y_data_test, y_pred_test_dt_data, pos_label='1'),
                                        recall_score(y_data_test, y_pred_test_dt_data, pos_label='1')]})

display(performance_dt_data)

Logistic Regression Classifier with DownSampling


,Error_metric,Train,Test
0,Accuracy,0.739799,0.719251
1,Precision,0.734063,0.705882
2,Recall,0.757475,0.725275



KNN Classifier with DownSampling


,Error_metric,Train,Test
0,Accuracy,0.840134,0.644385
1,Precision,1.000000,0.682836
2,Recall,0.682392,0.502747



Decision Tree Classifier with DownSampling


,Error_metric,Train,Test
0,Accuracy,0.730435,0.719251
1,Precision,0.694275,0.672646
2,Recall,0.829900,0.824176


UPSAMPLING

In [162]:
category_0 = churnData[churnData['Churn'] == "0"]
category_1 = churnData[churnData['Churn'] == "1"]

c0_len = len(category_0)
c1_len = len(category_1)

# upsample the positive class now
# - pump it up to the length of the negative class by allowing for row repetition
category_1_up = category_1.sample(c0_len, replace=True) # replace=True allows sampling of the same row more than once.
print(category_1_up.shape) # see that it's the same size a category_0

data_up = pd.concat([category_0, category_1_up], axis=0)
#shuffling the data
data_up = data_up.sample(frac=1)
data_up['Churn'].value_counts()

(5174, 16)


0    5174
1    5174
Name: Churn, dtype: int64

In [163]:
X_data = data_up[["tenure", "SeniorCitizen", "MonthlyCharges", "TotalCharges"]]
y_data = data_up["Churn"]

X_data_train, X_data_test, y_data_train, y_data_test = train_test_split(X_data, y_data, test_size=0.20, random_state=11)

scaler = StandardScaler()
# transform data
X_data_train = scaler.fit_transform(X_data_train)
X_data_test = scaler.transform(X_data_test) #No need to use fit since we should normalize the training set and then use the same normalization steps on the test set


In [164]:
log_model_data = LogisticRegression() 
#trans=PowerTransformer() #Optional transformation
#trans.fit(X_train_scaled)
#X_train_mod = trans.transform(X_train_scaled)
#X_test_mod  = trans.transform(X_test_scaled)
log_model.fit(X_data_train, y_data_train)
log_y_pred_test_data = log_model.predict(X_data_test) # predict test
log_y_pred_train_data = log_model.predict(X_data_train) # predict train (for sanity checks)

from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=2,weights='uniform') # declare we're using knn classification model
knn_model.fit(X_data_train, y_data_train) # train model
knn_y_pred_data = knn_model.predict(X_data_test) # predict test
knn_y_pred_train_data = knn_model.predict(X_data_train) # predict train (for sanity checks)
#model.predict_proba(inputdata)

from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier(max_depth=3) #Comes from sklearn.tree
dt_model.fit(X_data_train, y_data_train)
y_pred_train_dt_data = dt_model.predict(X_data_train)
y_pred_test_dt_data = dt_model.predict(X_data_test)

In [166]:
print("Logistic Regression Classifier with UpSampling")
performance_log_data = pd.DataFrame({'Error_metric': ['Accuracy','Precision','Recall'],
                               'Train': [accuracy_score(y_data_train, log_y_pred_train_data),
                                         precision_score(y_data_train, log_y_pred_train_data, pos_label='1'),
                                         recall_score(y_data_train, log_y_pred_train_data, pos_label='1')],
                               'Test': [accuracy_score(y_data_test, log_y_pred_test_data),
                                        precision_score(y_data_test, log_y_pred_test_data, pos_label='1'),
                                        recall_score(y_data_test, log_y_pred_test_data, pos_label='1')]})

display(performance_log_data)

print("\nKNN Classifier with UpSampling")

performance_knn_data = pd.DataFrame({'Error_metric': ['Accuracy','Precision','Recall'],
                               'Train': [accuracy_score(y_data_train, knn_y_pred_train_data),
                                         precision_score(y_data_train, knn_y_pred_train_data, pos_label='1'),
                                         recall_score(y_data_train, knn_y_pred_train_data, pos_label='1')],
                               'Test': [accuracy_score(y_data_test, knn_y_pred_data),
                                        precision_score(y_data_test, knn_y_pred_data, pos_label='1'),
                                        recall_score(y_data_test, knn_y_pred_data, pos_label='1')]})

display(performance_knn_data)

print("\nDecision Tree Classifier with UpSampling")

performance_dt_data = pd.DataFrame({'Error_metric': ['Accuracy','Precision','Recall'],
                               'Train': [accuracy_score(y_data_train, y_pred_train_dt_data),
                                         precision_score(y_data_train, y_pred_train_dt_data, pos_label='1'),
                                         recall_score(y_data_train, y_pred_train_dt_data, pos_label='1')],
                               'Test': [accuracy_score(y_data_test, y_pred_test_dt_data),
                                        precision_score(y_data_test, y_pred_test_dt_data, pos_label='1'),
                                        recall_score(y_data_test, y_pred_test_dt_data, pos_label='1')]})

display(performance_dt_data)

Logistic Regression Classifier with UpSampling


,Error_metric,Train,Test
0,Accuracy,0.736772,0.746860
1,Precision,0.733065,0.733086
2,Recall,0.746323,0.770204



KNN Classifier with UpSampling


,Error_metric,Train,Test
0,Accuracy,0.963518,0.828019
1,Precision,0.995873,0.834497
2,Recall,0.931034,0.814995



Decision Tree Classifier with UpSampling


,Error_metric,Train,Test
0,Accuracy,0.736651,0.747343
1,Precision,0.702575,0.706557
2,Recall,0.822522,0.839338
